In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import calendar
from time import time
import re 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import log_loss, accuracy_score, classification_report
%matplotlib inline

In [2]:
data_train = pd.read_csv("train.csv").replace("MENS WEAR","MENSWEAR")
data_train_orig = pd.read_csv("train.csv").replace("MENS WEAR","MENSWEAR")

In [3]:
dept_list = sorted(list(data_train.DepartmentDescription.dropna().unique()))

In [4]:
weekdays = list(calendar.day_name)
dept_list_sum = dict.fromkeys(dept_list, np.sum)
weekday_dict = dict.fromkeys(weekdays, np.max)
feature_dict = {"TripType": np.max, 'NumItems': np.sum, 'Return': np.max}
feature_dict = {**feature_dict, **weekday_dict, **dept_list_sum}

In [5]:
def transform_data(data):
    dummies = pd.get_dummies(data.Weekday)
    data[dummies.columns] = dummies
    
    dummies = pd.get_dummies(data.DepartmentDescription)
    dummies = dummies.apply(lambda x: x*data["ScanCount"])
    data[dummies.columns] = dummies 

    data.loc[data.ScanCount < 0, 'Return'] = 1
    data.loc[data.Return != 1, 'Return'] = 0
    
    data = data.rename(columns={"ScanCount":"NumItems"})
    
    grouped = data.groupby("VisitNumber")
    grouped = grouped.aggregate(feature_dict)
    data = grouped[["TripType", "NumItems", "Return"] + weekdays + dept_list]

    return data

In [6]:
data_new = transform_data(data_train)

In [7]:
def add_category_counts(data):
    alist = []
    for array in np.asarray(data.loc[:, dept_list[0]:]):
        count = 0
        count = sum(x > 0 for x in array)
        alist.append(count)
    cat_counts = pd.DataFrame(alist)
    cat_counts = cat_counts.rename(columns={0:"CategoryCount"})
    cat_counts = cat_counts.set_index(data.index)
    data.insert(3, 'CategoryCounts', cat_counts)
    return data

In [8]:
data_new_cat = add_category_counts(data_new)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils.np_utils import to_categorical
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD

Using TensorFlow backend.


In [10]:
scoring = ['neg_log_loss', 'accuracy']

In [11]:
X = data_new_cat.drop('TripType', axis=1)
y = pd.get_dummies(data_new_cat.TripType)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [12]:
type(X), type(y)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

In [13]:
y.shape

(95674, 38)

In [14]:
len(X.columns), X_train.shape, y_train.shape, X_test.shape, y_test.shape

(77, (71755, 77), (71755, 38), (23919, 77), (23919, 38))

In [15]:
network = Sequential()
# Add fully connected layer with a ReLU activation function
network.add(Dense(output_dim=128, activation="relu", input_shape=(len(X.columns),)))
# Add fully connected layer with a softmax activation function
network.add(Dense(output_dim=len(y.columns), activation="softmax"))
network.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

W0831 23:26:14.564530 19696 deprecation_wrapper.py:119] From C:\Users\Xiuquan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_shape=(77,), units=128)`
  This is separate from the ipykernel package so we can avoid doing imports until
W0831 23:26:14.626494 19696 deprecation_wrapper.py:119] From C:\Users\Xiuquan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0831 23:26:14.635489 19696 deprecation_wrapper.py:119] From C:\Users\Xiuquan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

C:\Users\Xiuquan\Anaconda3\li

In [16]:
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               9984      
_________________________________________________________________
dense_2 (Dense)              (None, 38)                4902      
Total params: 14,886
Trainable params: 14,886
Non-trainable params: 0
_________________________________________________________________


In [17]:
checkpoint = [ModelCheckpoint(filepath="nn_model.hdf5")]

In [19]:
history = network.fit(X_train, y_train, epochs=90, callbacks=checkpoint, verbose=2, batch_size=5000, validation_data=(X_test, y_test))

Train on 71755 samples, validate on 23919 samples
Epoch 1/90
 - 1s - loss: 0.8746 - acc: 0.7059 - val_loss: 0.9072 - val_acc: 0.6997
Epoch 2/90
 - 1s - loss: 0.8726 - acc: 0.7061 - val_loss: 0.9100 - val_acc: 0.6984
Epoch 3/90
 - 1s - loss: 0.8712 - acc: 0.7065 - val_loss: 0.9043 - val_acc: 0.7006
Epoch 4/90
 - 1s - loss: 0.8676 - acc: 0.7081 - val_loss: 0.9059 - val_acc: 0.7005
Epoch 5/90
 - 1s - loss: 0.8668 - acc: 0.7067 - val_loss: 0.9001 - val_acc: 0.7020
Epoch 6/90
 - 1s - loss: 0.8632 - acc: 0.7079 - val_loss: 0.8993 - val_acc: 0.7004
Epoch 7/90
 - 1s - loss: 0.8621 - acc: 0.7075 - val_loss: 0.9016 - val_acc: 0.6987
Epoch 8/90
 - 1s - loss: 0.8595 - acc: 0.7093 - val_loss: 0.9008 - val_acc: 0.6999
Epoch 9/90
 - 1s - loss: 0.8577 - acc: 0.7091 - val_loss: 0.8987 - val_acc: 0.7002
Epoch 10/90
 - 1s - loss: 0.8564 - acc: 0.7090 - val_loss: 0.9029 - val_acc: 0.6985
Epoch 11/90
 - 1s - loss: 0.8537 - acc: 0.7099 - val_loss: 0.8968 - val_acc: 0.6997
Epoch 12/90
 - 1s - loss: 0.8534 - 

In [32]:
network2 = Sequential()
# Add fully connected layer with a ReLU activation function
network2.add(Dense(output_dim=128, activation="relu", input_shape=(len(X.columns),)))
network2.add(Dense(output_dim=64, activation="relu"))
# Add fully connected layer with a softmax activation function
network2.add(Dense(output_dim=len(y.columns), activation="softmax"))
network2.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_shape=(77,), units=128)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=64)`
  after removing the cwd from sys.path.
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  


In [33]:
network2.fit(X_train, y_train, epochs=90, callbacks=checkpoint, verbose=2, batch_size=5000, validation_data=(X_test, y_test))

Train on 71755 samples, validate on 23919 samples
Epoch 1/90
 - 2s - loss: 3.2055 - acc: 0.2303 - val_loss: 2.7921 - val_acc: 0.4161
Epoch 2/90
 - 1s - loss: 2.6014 - acc: 0.4352 - val_loss: 2.3663 - val_acc: 0.4585
Epoch 3/90
 - 1s - loss: 2.1873 - acc: 0.5018 - val_loss: 1.9895 - val_acc: 0.5157
Epoch 4/90
 - 1s - loss: 1.8641 - acc: 0.5510 - val_loss: 1.7097 - val_acc: 0.5663
Epoch 5/90
 - 1s - loss: 1.6532 - acc: 0.5793 - val_loss: 1.5310 - val_acc: 0.6019
Epoch 6/90
 - 1s - loss: 1.4910 - acc: 0.6154 - val_loss: 1.4108 - val_acc: 0.6289
Epoch 7/90
 - 1s - loss: 1.3748 - acc: 0.6320 - val_loss: 1.3200 - val_acc: 0.6284
Epoch 8/90
 - 1s - loss: 1.2957 - acc: 0.6430 - val_loss: 1.2479 - val_acc: 0.6476
Epoch 9/90
 - 1s - loss: 1.2368 - acc: 0.6509 - val_loss: 1.2276 - val_acc: 0.6518
Epoch 10/90
 - 1s - loss: 1.1813 - acc: 0.6630 - val_loss: 1.1860 - val_acc: 0.6457
Epoch 11/90
 - 1s - loss: 1.1501 - acc: 0.6647 - val_loss: 1.1299 - val_acc: 0.6704
Epoch 12/90
 - 1s - loss: 1.1235 - 

In [34]:
network3 = Sequential()
# Add fully connected layer with a ReLU activation function
network3.add(Dense(output_dim=256, activation="relu", input_shape=(len(X.columns),)))
network3.add(Dropout(0.3))
network3.add(Dense(output_dim=128, activation="relu"))
network3.add(Dropout(0.3))
network3.add(Dense(output_dim=64, activation="relu"))
network3.add(Dropout(0.3))
# Add fully connected layer with a softmax activation function
network3.add(Dense(output_dim=len(y.columns), activation="softmax"))
network3.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_shape=(77,), units=256)`
  This is separate from the ipykernel package so we can avoid doing imports until
W0831 23:58:44.143802 19696 deprecation.py:506] From C:\Users\Xiuquan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3733: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=64)`
  import sys
C:\Use

In [35]:
network3.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 256)               19968     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 38)               

In [36]:
start = time()
network3.fit(X_train, y_train, epochs=90, callbacks=checkpoint, verbose=2, batch_size=5000, validation_data=(X_test, y_test))
end = time()
end-start

Train on 71755 samples, validate on 23919 samples
Epoch 1/90
 - 4s - loss: 3.2457 - acc: 0.1857 - val_loss: 2.6140 - val_acc: 0.3359
Epoch 2/90
 - 3s - loss: 2.5965 - acc: 0.3300 - val_loss: 2.0520 - val_acc: 0.4876
Epoch 3/90
 - 3s - loss: 2.1555 - acc: 0.4262 - val_loss: 1.6569 - val_acc: 0.5810
Epoch 4/90
 - 3s - loss: 1.8957 - acc: 0.4931 - val_loss: 1.4546 - val_acc: 0.6201
Epoch 5/90
 - 3s - loss: 1.7265 - acc: 0.5376 - val_loss: 1.3388 - val_acc: 0.6401
Epoch 6/90
 - 3s - loss: 1.6078 - acc: 0.5650 - val_loss: 1.2682 - val_acc: 0.6449
Epoch 7/90
 - 3s - loss: 1.5331 - acc: 0.5825 - val_loss: 1.2122 - val_acc: 0.6584
Epoch 8/90
 - 4s - loss: 1.4693 - acc: 0.5968 - val_loss: 1.1612 - val_acc: 0.6652
Epoch 9/90
 - 5s - loss: 1.4209 - acc: 0.6034 - val_loss: 1.1484 - val_acc: 0.6673
Epoch 10/90
 - 6s - loss: 1.3792 - acc: 0.6155 - val_loss: 1.1037 - val_acc: 0.6776
Epoch 11/90
 - 6s - loss: 1.3487 - acc: 0.6176 - val_loss: 1.0844 - val_acc: 0.6771
Epoch 12/90
 - 6s - loss: 1.3119 - 

530.3734087944031

In [37]:
start = time()
network3.fit(X_train, y_train, epochs=90, callbacks=checkpoint, verbose=2, batch_size=1000, validation_data=(X_test, y_test))
end = time()
end-start

Train on 71755 samples, validate on 23919 samples
Epoch 1/90
 - 7s - loss: 0.9651 - acc: 0.6900 - val_loss: 0.8740 - val_acc: 0.7089
Epoch 2/90
 - 7s - loss: 0.9596 - acc: 0.6929 - val_loss: 0.8744 - val_acc: 0.7099
Epoch 3/90
 - 7s - loss: 0.9581 - acc: 0.6919 - val_loss: 0.8765 - val_acc: 0.7088
Epoch 4/90
 - 7s - loss: 0.9556 - acc: 0.6926 - val_loss: 0.8765 - val_acc: 0.7085
Epoch 5/90
 - 7s - loss: 0.9533 - acc: 0.6931 - val_loss: 0.8729 - val_acc: 0.7081
Epoch 6/90
 - 7s - loss: 0.9488 - acc: 0.6932 - val_loss: 0.8728 - val_acc: 0.7091
Epoch 7/90
 - 7s - loss: 0.9484 - acc: 0.6948 - val_loss: 0.8735 - val_acc: 0.7082
Epoch 8/90
 - 7s - loss: 0.9481 - acc: 0.6939 - val_loss: 0.8750 - val_acc: 0.7084
Epoch 9/90
 - 7s - loss: 0.9449 - acc: 0.6943 - val_loss: 0.8701 - val_acc: 0.7094
Epoch 10/90
 - 7s - loss: 0.9436 - acc: 0.6951 - val_loss: 0.8697 - val_acc: 0.7103
Epoch 11/90
 - 6s - loss: 0.9459 - acc: 0.6956 - val_loss: 0.8682 - val_acc: 0.7101
Epoch 12/90
 - 7s - loss: 0.9407 - 

613.0093688964844

In [38]:
start = time()
network3.fit(X_train, y_train, epochs=90, callbacks=checkpoint, verbose=2, batch_size=500, validation_data=(X_test, y_test))
end = time()
end-start

Train on 71755 samples, validate on 23919 samples
Epoch 1/90
 - 3s - loss: 0.8767 - acc: 0.7089 - val_loss: 0.8713 - val_acc: 0.7125
Epoch 2/90
 - 4s - loss: 0.8757 - acc: 0.7100 - val_loss: 0.8701 - val_acc: 0.7108
Epoch 3/90
 - 5s - loss: 0.8803 - acc: 0.7083 - val_loss: 0.8728 - val_acc: 0.7085
Epoch 4/90
 - 5s - loss: 0.8814 - acc: 0.7096 - val_loss: 0.8725 - val_acc: 0.7093
Epoch 5/90
 - 4s - loss: 0.8795 - acc: 0.7093 - val_loss: 0.8727 - val_acc: 0.7109
Epoch 6/90
 - 4s - loss: 0.8834 - acc: 0.7093 - val_loss: 0.8729 - val_acc: 0.7080
Epoch 7/90
 - 4s - loss: 0.8816 - acc: 0.7086 - val_loss: 0.8723 - val_acc: 0.7117
Epoch 8/90
 - 4s - loss: 0.8826 - acc: 0.7106 - val_loss: 0.8771 - val_acc: 0.7089
Epoch 9/90
 - 4s - loss: 0.8818 - acc: 0.7092 - val_loss: 0.8721 - val_acc: 0.7111
Epoch 10/90
 - 4s - loss: 0.8814 - acc: 0.7090 - val_loss: 0.8717 - val_acc: 0.7099
Epoch 11/90
 - 4s - loss: 0.8827 - acc: 0.7071 - val_loss: 0.8751 - val_acc: 0.7083
Epoch 12/90
 - 4s - loss: 0.8843 - 

578.317896604538

In [43]:
network3 = Sequential()
# Add fully connected layer with a ReLU activation function
network3.add(Dense(output_dim=256, activation="relu", input_shape=(len(X.columns),)))
network3.add(Dropout(0.3))
network3.add(Dense(output_dim=128, activation="relu"))
network3.add(Dropout(0.3))
network3.add(Dense(output_dim=64, activation="relu"))
network3.add(Dropout(0.3))
# Add fully connected layer with a softmax activation function
network3.add(Dense(output_dim=len(y.columns), activation="softmax"))
network3.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_shape=(77,), units=256)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=64)`
  import sys
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  # Remove the CWD from sys.path while we load stuff.


In [44]:
start = time()
network3.fit(X_train, y_train, epochs=90, callbacks=checkpoint, verbose=2, batch_size=1000, validation_data=(X_test, y_test))
end = time()
end-start

Train on 71755 samples, validate on 23919 samples
Epoch 1/90
 - 5s - loss: 2.8336 - acc: 0.2582 - val_loss: 1.8253 - val_acc: 0.5462
Epoch 2/90
 - 4s - loss: 1.7795 - acc: 0.5219 - val_loss: 1.2818 - val_acc: 0.6500
Epoch 3/90
 - 3s - loss: 1.4730 - acc: 0.5950 - val_loss: 1.1377 - val_acc: 0.6729
Epoch 4/90
 - 3s - loss: 1.3467 - acc: 0.6177 - val_loss: 1.0756 - val_acc: 0.6768
Epoch 5/90
 - 3s - loss: 1.2750 - acc: 0.6324 - val_loss: 1.0420 - val_acc: 0.6852
Epoch 6/90
 - 4s - loss: 1.2368 - acc: 0.6403 - val_loss: 1.0127 - val_acc: 0.6888
Epoch 7/90
 - 4s - loss: 1.2087 - acc: 0.6454 - val_loss: 0.9969 - val_acc: 0.6900
Epoch 8/90
 - 3s - loss: 1.1789 - acc: 0.6501 - val_loss: 0.9868 - val_acc: 0.6923
Epoch 9/90
 - 3s - loss: 1.1606 - acc: 0.6550 - val_loss: 0.9761 - val_acc: 0.6924
Epoch 10/90
 - 3s - loss: 1.1458 - acc: 0.6537 - val_loss: 0.9645 - val_acc: 0.6926
Epoch 11/90
 - 3s - loss: 1.1283 - acc: 0.6586 - val_loss: 0.9555 - val_acc: 0.6971
Epoch 12/90
 - 4s - loss: 1.1141 - 

535.0121166706085

In [45]:
network3 = Sequential()
# Add fully connected layer with a ReLU activation function
network3.add(Dense(output_dim=256, activation="relu", input_shape=(len(X.columns),)))
network3.add(Dropout(0.3))
network3.add(Dense(output_dim=128, activation="relu"))
network3.add(Dropout(0.3))
network3.add(Dense(output_dim=64, activation="relu"))
network3.add(Dropout(0.3))
# Add fully connected layer with a softmax activation function
network3.add(Dense(output_dim=len(y.columns), activation="softmax"))
network3.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_shape=(77,), units=256)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=64)`
  import sys
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  # Remove the CWD from sys.path while we load stuff.


In [46]:
start = time()
network3.fit(X_train, y_train, epochs=90, callbacks=checkpoint, verbose=2, batch_size=1000, validation_data=(X_test, y_test))
end = time()
end-start

Train on 71755 samples, validate on 23919 samples
Epoch 1/90
 - 6s - loss: 3.5988 - acc: 0.0618 - val_loss: 3.3380 - val_acc: 0.0826
Epoch 2/90
 - 3s - loss: 3.3995 - acc: 0.1086 - val_loss: 3.2506 - val_acc: 0.1320
Epoch 3/90
 - 4s - loss: 3.3126 - acc: 0.1499 - val_loss: 3.1737 - val_acc: 0.2042
Epoch 4/90
 - 4s - loss: 3.2419 - acc: 0.1878 - val_loss: 3.1027 - val_acc: 0.2492
Epoch 5/90
 - 4s - loss: 3.1742 - acc: 0.2167 - val_loss: 3.0296 - val_acc: 0.2868
Epoch 6/90
 - 3s - loss: 3.1081 - acc: 0.2394 - val_loss: 2.9527 - val_acc: 0.3070
Epoch 7/90
 - 4s - loss: 3.0407 - acc: 0.2550 - val_loss: 2.8727 - val_acc: 0.3170
Epoch 8/90
 - 4s - loss: 2.9680 - acc: 0.2669 - val_loss: 2.7916 - val_acc: 0.3241
Epoch 9/90
 - 4s - loss: 2.8968 - acc: 0.2755 - val_loss: 2.7118 - val_acc: 0.3269
Epoch 10/90
 - 4s - loss: 2.8243 - acc: 0.2841 - val_loss: 2.6337 - val_acc: 0.3301
Epoch 11/90
 - 4s - loss: 2.7579 - acc: 0.2905 - val_loss: 2.5622 - val_acc: 0.3332
Epoch 12/90
 - 3s - loss: 2.6983 - 

499.7338709831238

In [47]:
def create_network():
    network3 = Sequential()
    # Add fully connected layer with a ReLU activation function
    network3.add(Dense(output_dim=256, activation="relu", input_shape=(len(X.columns),)))
    network3.add(Dropout(0.3))
    network3.add(Dense(output_dim=128, activation="relu"))
    network3.add(Dropout(0.3))
    network3.add(Dense(output_dim=64, activation="relu"))
    network3.add(Dropout(0.3))
    # Add fully connected layer with a softmax activation function
    network3.add(Dense(output_dim=len(y.columns), activation="softmax"))
    network3.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    
    return network3

In [48]:
neural_network = KerasClassifier(build_fn=create_network, epochs=50, batch_size=1000, verbose=2)

In [49]:
start = time()
cross_val_score(neural_network, X, y, cv=4)
end = time()
end-start

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_shape=(77,), units=256)`
  after removing the cwd from sys.path.
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=64)`
  
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/50
 - 11s - loss: 2.6277 - acc: 0.3107
Epoch 2/50
 - 3s - loss: 1.7243 - acc: 0.5397
Epoch 3/50
 - 4s - loss: 1.4706 - acc: 0.5945
Epoch 4/50
 - 5s - loss: 1.3478 - acc: 0.6222
Epoch 5/50
 - 4s - loss: 1.2786 - acc: 0.6320
Epoch 6/50
 - 4s - loss: 1.2361 - acc: 0.6412
Epoch 7/50
 - 5s - loss: 1.2062 - acc: 0.6459
Epoch 8/50
 - 5s - loss: 1.1759 - acc: 0.6511
Epoch 9/50
 - 4s - loss: 1.1539 - acc: 0.6559
Epoch 10/50
 - 5s - loss: 1.1368 - acc: 0.6573
Epoch 11/50
 - 5s - loss: 1.1296 - acc: 0.6582
Epoch 12/50
 - 5s - loss: 1.1099 - acc: 0.6628
Epoch 13/50
 - 5s - loss: 1.0962 - acc: 0.6666
Epoch 14/50
 - 4s - loss: 1.0879 - acc: 0.6696
Epoch 15/50
 - 5s - loss: 1.0752 - acc: 0.6706
Epoch 16/50
 - 5s - loss: 1.0683 - acc: 0.6711
Epoch 17/50
 - 4s - loss: 1.0591 - acc: 0.6749
Epoch 18/50
 - 4s - loss: 1.0487 - acc: 0.6774
Epoch 19/50
 - 5s - loss: 1.0482 - acc: 0.6749
Epoch 20/50
 - 6s - loss: 1.0388 - acc: 0.6761
Epoch 21/50
 - 6s - loss: 1.0321 - acc: 0.6783
Epoch 22/50
 - 5s - l

Epoch 26/50
 - 5s - loss: 1.0020 - acc: 0.6853
Epoch 27/50
 - 5s - loss: 0.9977 - acc: 0.6842
Epoch 28/50
 - 5s - loss: 0.9986 - acc: 0.6850
Epoch 29/50
 - 5s - loss: 0.9939 - acc: 0.6855
Epoch 30/50
 - 5s - loss: 0.9889 - acc: 0.6875
Epoch 31/50
 - 5s - loss: 0.9850 - acc: 0.6876
Epoch 32/50
 - 5s - loss: 0.9813 - acc: 0.6896
Epoch 33/50
 - 5s - loss: 0.9803 - acc: 0.6897
Epoch 34/50
 - 5s - loss: 0.9709 - acc: 0.6905
Epoch 35/50
 - 5s - loss: 0.9722 - acc: 0.6909
Epoch 36/50
 - 5s - loss: 0.9685 - acc: 0.6916
Epoch 37/50
 - 5s - loss: 0.9673 - acc: 0.6927
Epoch 38/50
 - 5s - loss: 0.9637 - acc: 0.6924
Epoch 39/50
 - 5s - loss: 0.9622 - acc: 0.6921
Epoch 40/50
 - 5s - loss: 0.9600 - acc: 0.6923
Epoch 41/50
 - 5s - loss: 0.9571 - acc: 0.6922
Epoch 42/50
 - 5s - loss: 0.9520 - acc: 0.6958
Epoch 43/50
 - 5s - loss: 0.9507 - acc: 0.6936
Epoch 44/50
 - 5s - loss: 0.9490 - acc: 0.6949
Epoch 45/50
 - 5s - loss: 0.9473 - acc: 0.6951
Epoch 46/50
 - 5s - loss: 0.9490 - acc: 0.6944
Epoch 47/50
 

1104.957437992096

In [53]:
def create_network():
    network3 = Sequential()
    # Add fully connected layer with a ReLU activation function
    network3.add(Dense(output_dim=256, kernel_regularizer=regularizers.l2(0.01), activation="relu", input_dim=len(X.columns)))
    network3.add(Dropout(0.3))
    network3.add(Dense(output_dim=128, kernel_regularizer=regularizers.l2(0.01), activation="relu"))
    network3.add(Dropout(0.3))
    network3.add(Dense(output_dim=64, kernel_regularizer=regularizers.l2(0.01), activation="relu"))
    network3.add(Dropout(0.3))
    # Add fully connected layer with a softmax activation function
    network3.add(Dense(output_dim=len(y.columns), activation="softmax"))
    network3.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    
    return network3

In [54]:
neural_network = KerasClassifier(build_fn=create_network, epochs=100, batch_size=1000, verbose=2)

In [55]:
start = time()
cross_val_score(neural_network, X, y, cv=4)
end = time()
end-start

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_regularizer=<keras.reg..., activation="relu", input_dim=77, units=256)`
  after removing the cwd from sys.path.
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_regularizer=<keras.reg..., activation="relu", units=128)`
  
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_regularizer=<keras.reg..., activation="relu", units=64)`
  
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/100
 - 10s - loss: 4.8007 - acc: 0.2636
Epoch 2/100
 - 7s - loss: 2.8632 - acc: 0.4330
Epoch 3/100
 - 6s - loss: 2.3906 - acc: 0.5087
Epoch 4/100
 - 6s - loss: 2.2071 - acc: 0.5342
Epoch 5/100
 - 6s - loss: 2.1067 - acc: 0.5488
Epoch 6/100
 - 6s - loss: 2.0361 - acc: 0.5612
Epoch 7/100
 - 6s - loss: 1.9882 - acc: 0.5682
Epoch 8/100
 - 6s - loss: 1.9563 - acc: 0.5734
Epoch 9/100
 - 5s - loss: 1.9198 - acc: 0.5800
Epoch 10/100
 - 5s - loss: 1.8973 - acc: 0.5802
Epoch 11/100
 - 5s - loss: 1.8715 - acc: 0.5867
Epoch 12/100
 - 5s - loss: 1.8588 - acc: 0.5888
Epoch 13/100
 - 5s - loss: 1.8364 - acc: 0.5922
Epoch 14/100
 - 5s - loss: 1.8201 - acc: 0.5937
Epoch 15/100
 - 6s - loss: 1.8095 - acc: 0.5952
Epoch 16/100
 - 6s - loss: 1.7904 - acc: 0.6013
Epoch 17/100
 - 5s - loss: 1.7802 - acc: 0.6034
Epoch 18/100
 - 5s - loss: 1.7651 - acc: 0.6061
Epoch 19/100
 - 6s - loss: 1.7547 - acc: 0.6050
Epoch 20/100
 - 5s - loss: 1.7452 - acc: 0.6057
Epoch 21/100
 - 5s - loss: 1.7363 - acc: 0.6086


Epoch 72/100
 - 5s - loss: 1.5092 - acc: 0.6445
Epoch 73/100
 - 5s - loss: 1.5039 - acc: 0.6464
Epoch 74/100
 - 5s - loss: 1.4968 - acc: 0.6472
Epoch 75/100
 - 5s - loss: 1.4978 - acc: 0.6485
Epoch 76/100
 - 5s - loss: 1.4964 - acc: 0.6469
Epoch 77/100
 - 5s - loss: 1.4937 - acc: 0.6495
Epoch 78/100
 - 5s - loss: 1.4986 - acc: 0.6463
Epoch 79/100
 - 5s - loss: 1.4912 - acc: 0.6469
Epoch 80/100
 - 5s - loss: 1.4910 - acc: 0.6465
Epoch 81/100
 - 5s - loss: 1.4868 - acc: 0.6489
Epoch 82/100
 - 5s - loss: 1.4854 - acc: 0.6481
Epoch 83/100
 - 6s - loss: 1.4809 - acc: 0.6492
Epoch 84/100
 - 6s - loss: 1.4820 - acc: 0.6489
Epoch 85/100
 - 6s - loss: 1.4829 - acc: 0.6499
Epoch 86/100
 - 6s - loss: 1.4798 - acc: 0.6507
Epoch 87/100
 - 6s - loss: 1.4783 - acc: 0.6502
Epoch 88/100
 - 5s - loss: 1.4787 - acc: 0.6508
Epoch 89/100
 - 6s - loss: 1.4765 - acc: 0.6498
Epoch 90/100
 - 5s - loss: 1.4721 - acc: 0.6501
Epoch 91/100
 - 5s - loss: 1.4763 - acc: 0.6499
Epoch 92/100
 - 5s - loss: 1.4689 - acc:

Epoch 43/100
 - 7s - loss: 1.6018 - acc: 0.6271
Epoch 44/100
 - 8s - loss: 1.6001 - acc: 0.6295
Epoch 45/100
 - 7s - loss: 1.5959 - acc: 0.6286
Epoch 46/100
 - 7s - loss: 1.5884 - acc: 0.6299
Epoch 47/100
 - 7s - loss: 1.5889 - acc: 0.6308
Epoch 48/100
 - 7s - loss: 1.5846 - acc: 0.6305
Epoch 49/100
 - 8s - loss: 1.5837 - acc: 0.6299
Epoch 50/100
 - 7s - loss: 1.5776 - acc: 0.6318
Epoch 51/100
 - 7s - loss: 1.5745 - acc: 0.6318
Epoch 52/100
 - 7s - loss: 1.5749 - acc: 0.6322
Epoch 53/100
 - 7s - loss: 1.5686 - acc: 0.6335
Epoch 54/100
 - 7s - loss: 1.5683 - acc: 0.6330
Epoch 55/100
 - 7s - loss: 1.5636 - acc: 0.6338
Epoch 56/100
 - 7s - loss: 1.5638 - acc: 0.6321
Epoch 57/100
 - 7s - loss: 1.5624 - acc: 0.6353
Epoch 58/100
 - 7s - loss: 1.5553 - acc: 0.6370
Epoch 59/100
 - 7s - loss: 1.5542 - acc: 0.6344
Epoch 60/100
 - 7s - loss: 1.5538 - acc: 0.6360
Epoch 61/100
 - 7s - loss: 1.5536 - acc: 0.6341
Epoch 62/100
 - 7s - loss: 1.5491 - acc: 0.6353
Epoch 63/100
 - 7s - loss: 1.5489 - acc:

2398.770131587982

In [56]:
len(X.columns)

77